In [1]:
### Required libraries ###

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dropout, Dense, Input, BatchNormalization, Activation, Add, LSTM, Softmax, Bidirectional, Conv1D
from keras.optimizers import Adam
from keras.utils import to_categorical, normalize
from keras import backend as K

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### Dataset ###

# reading dataset
dataset = pd.read_csv(r"D:\Data\Dinesh\Work\DLTraining\UCI Credit Card - Demographics Model\UCI_Credit_Card.csv", index_col=0)
dataset["AGE_CAT"] = pd.cut(dataset["AGE"], [0, 15, 25, 35, 45, 55, 80], labels=[15, 25, 35, 45, 55, 80])

neg_num_categorical_columns = ["PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
for column in neg_num_categorical_columns:
    min_value = abs(dataset[column].min())
    dataset[column] += abs(min_value)

In [3]:
### Data Preprocessing ###

X_payment = normalize(np.array(dataset[:, 17:22], dtype="float32").reshape(30000, 5, 1))
Y_payment = normalize(np.array(dataset[:, 22], dtype="float32").reshape(30000, 1))

X_default = normalize(np.array(dataset[:, 17:23], dtype="float32").reshape(30000 , 6, 1))
Y_default = to_categorical(np.array(dataset[:, 23], dtype="float32")).reshape(30000, 2)

X_seq = X_payment_pred
Y_seq = Y_payment_pred
m_seq = X_seq.shape[0]
n_X_seq = X_seq.shape[1]
n_X_seq_el = X_seq.shape[2]
n_Y_seq = Y_seq.shape[1]

(30000, 71) (30000, 3) (30000, 7) (30000, 4) (30000, 1) (30000, 81)


In [ ]:
### Train and Test set preparation ###

# Splitting into test, train and dev set for each of the demographic label. All demographic labels can also be kept as single Y
X_temp, X_test, Y_education_temp, Y_education_test = train_test_split(X, Y_education, test_size=0.10, random_state=5)
X_train, X_dev, Y_education_train, Y_education_dev = train_test_split(X, Y_education, test_size=0.10, random_state=5)
X_temp, X_test, Y_sex_temp, Y_sex_test = train_test_split(X, Y_sex, test_size=0.10, random_state=5)
X_train, X_dev, Y_sex_train, Y_sex_dev = train_test_split(X, Y_sex, test_size=0.10, random_state=5)
X_temp, X_test, Y_marriage_temp, Y_marriage_test = train_test_split(X, Y_marriage, test_size=0.10, random_state=5)
X_train, X_dev, Y_marriage_train, Y_marriage_dev = train_test_split(X, Y_marriage, test_size=0.10, random_state=5)
X_temp, X_test, Y_age_temp, Y_age_test = train_test_split(X, Y_age, test_size=0.10, random_state=5)
X_train, X_dev, Y_age_train, Y_age_dev = train_test_split(X, Y_age, test_size=0.10, random_state=5)
X_temp, X_test, Y_age_cat_temp, Y_age_cat_test = train_test_split(X, Y_age_cat, test_size=0.10, random_state=5)
X_train, X_dev, Y_age_cat_train, Y_age_cat_dev = train_test_split(X, Y_age_cat, test_size=0.10, random_state=5)
'''
Note: Final datasets for 'education'(modify above two lines for any other demography) 
            - X_train, Y_education_train, X_dev, Y_education_dev, X_test, Y_education_test
'''
None

In [5]:
### Model ###

def build_model(layers_dims, n_X, n_Y):
    
    # input layer
    X_input = Input(shape = (n_X,))
    A = X_input

    # hidden layer
    # list having no. of activations in each layer
    add_this_a = None
    fix_add_this_a = False
#     layers_dims = [8, 12, 16, 16, 12, 8]
    for i, dim in enumerate(layers_dims):

        layer_no = i + 1

#         skip_add = True if (layer_no%3 == 0 and layer_no/3 > 1) else False # for skip connections, every 3 layers
#         add_this_A = True if (layer_no + 2)%3 == 0 else False # for skip connections

        A = Dense(dim, activation = "linear")(A) # to change activation check here, https://keras.io/activations/

#         if skip_add == True: # for skip connections
#             Add()([A, to_add_A])

        A = Activation("relu")(A)

#         if add_this_A == True: # for skip connections
#             to_add_A = A

        A = BatchNormalization()(A)
    #     A = Dropout(0.7)(A) # add dropout, if needed


    Y_layer = Dense(n_Y, activation="softmax")(A)
    # Y_layer = Dense(1, activation="linear")(A)

    model = Model(inputs = [X_input], outputs = [Y_layer])

    # adam optimizer - try different learning rates here
    opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    
    return model

# prints model graph
# model.summary()

In [6]:
# X and Y for the current run, and their dimensions
X = X_train
Y = Y_education_train
m = X.shape[0] # no. of training examples
n_X = X.shape[1] # no. of features
n_Y = Y.shape[1] # no. of output classes

# clearing session
K.clear_session()

# build model with a set of dimensions
model = build_model([], n_X, n_Y)
# print out the model layers
# model.summary()

# training model
model.fit(X, Y, epochs=5, batch_size=64)
model.evaluate(X_test, Y_education_test)

Epoch 1/5
27000/27000 [==============================] - 0s 16us/step - loss: 1.1482 - acc: 0.4994 
Epoch 2/5
27000/27000 [==============================] - 0s 11us/step - loss: 1.0783 - acc: 0.5067

[1.0460636178652445, 0.5233333331743876]

In [7]:
# X and Y for the current run, and their dimensions
X = X_train
Y = Y_sex_train
m = X.shape[0] # no. of training examples
n_X = X.shape[1] # no. of features
n_Y = Y.shape[1] # no. of output classes

# clearing session
K.clear_session()

# build model with a set of dimensions
model = build_model([], n_X, n_Y)
# print out the model layers
# model.summary()

# training model
model.fit(X, Y, epochs=5, batch_size=64)
model.evaluate(X_test, Y_sex_test)

Epoch 1/5
27000/27000 [==============================] - 0s 14us/step - loss: 0.7116 - acc: 0.5980
Epoch 2/5
27000/27000 [==============================] - 0s 11us/step - loss: 0.6723 - acc: 0.6053

[0.6714541255633036, 0.6023333333333334]

In [8]:
# X and Y for the current run, and their dimensions
X = X_train
Y = Y_marriage_train
m = X.shape[0] # no. of training examples
n_X = X.shape[1] # no. of features
n_Y = Y.shape[1] # no. of output classes

# clearing session
K.clear_session()

# build model with a set of dimensions
model = build_model([], n_X, n_Y)
# print out the model layers
# model.summary()

# training model
model.fit(X, Y, epochs=5, batch_size=64)
model.evaluate(X_test, Y_marriage_test)

Epoch 1/5
27000/27000 [==============================] - 0s 14us/step - loss: 0.7973 - acc: 0.5261
Epoch 2/5
27000/27000 [==============================] - 0s 10us/step - loss: 0.7549 - acc: 0.5340

[0.739679926554362, 0.5573333331743876]

In [9]:
# X and Y for the current run, and their dimensions
X = X_train
Y = Y_age_cat_train
m = X.shape[0] # no. of training examples
n_X = X.shape[1] # no. of features
n_Y = Y.shape[1] # no. of output classes

# clearing session
K.clear_session()

# build model with a set of dimensions
model = build_model([], n_X, n_Y)
# print out the model layers
# model.summary()

# training model
model.fit(X, Y, epochs=5, batch_size=64)
model.evaluate(X_test, Y_age_cat_test)

Epoch 1/5
27000/27000 [==============================] - 0s 17us/step - loss: 1.6965 - acc: 0.4006
Epoch 2/5
27000/27000 [==============================] - 0s 14us/step - loss: 1.3486 - acc: 0.4324

[1.3570076888402303, 0.42000000007947286]